<a href="https://colab.research.google.com/github/runnin123/Jeju_Bigdata/blob/master/jeju_diff_corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from itertools import product

# 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 통계
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
class Model:
    # Initialization method of the Model class about definition of the data
    # Input : split_data(Training/Test set of the independent value and the dependent value)
    #       : num(encoding_data)
    # Definition of the X
    def __init__(self, split_data, num):
        self.X_train = split_data[0]
        self.X_test = split_data[1]
        self.y_train = split_data[2]
        self.y_test = split_data[3]
        self.encoding_data = num

    # the rmsle between the actual value and predicted value
    def rmsle(self, y, pred): 
        log_y = np.log1p(y)
        log_pred = np.log1p(pred)
        squared_error = (log_y - log_pred)**2
        rmsle = np.sqrt(np.mean(squared_error))
        print('Test Data RMSLE: {0:.3f}'.format(rmsle))
        return round(rmsle, 3)

    # get best estimator and rmsle through GridSearchCV
    def best_params_model(self, model, params):
        cv_model = GridSearchCV(model, param_grid=params, scoring="neg_mean_squared_error", cv = 5)
        cv_model.fit(self.X_train, self.y_train)
        print("----", model.__class__.__name__, "----")
        print("GridSearchCV 최적 하이퍼 파라미터 :", cv_model.best_params_)

        eval_pred = cv_model.predict(self.X_test)
        eval_pred = np.expm1(eval_pred)
        rmsle_ = self.rmsle(self.y_test, eval_pred)

        return  cv_model.best_estimator_, rmsle_

    # Process about finding the ratio of XGB and GBM and LGB
    def get_model(self):
        xgb = XGBRegressor(random_state=0)
        gbm = GradientBoostingRegressor(random_state=0)
        lgb = LGBMRegressor(random_state=0)

        params = {'n_estimators': [1000, 2000]}

        models = [xgb, gbm, lgb]
        best_models = []
        rmsles = []
        
        for model in models:
            new_model = self.best_params_model(model, params)[0]
            new_rmsle = self.best_params_model(model, params)[1]
            best_models.append(new_model)
            rmsles.append(new_rmsle)

        self.xgb_reg = best_models[0]
        self.gbm_reg = best_models[1]
        self.lgb_reg = best_models[2]

        self.xgb_rmsle = rmsles[0]
        self.gbm_rmsle = rmsles[1]
        self.lgb_rmsle = rmsles[2]

        xgb_pred = self.xgb_reg.predict(self.X_test)
        xgb_pred = np.expm1(xgb_pred)

        gbm_pred = self.gbm_reg.predict(self.X_test)
        gbm_pred = np.expm1(gbm_pred)

        lgb_pred = self.lgb_reg.predict(self.X_test)
        lgb_pred = np.expm1(lgb_pred)

        pred = np.array([xgb_pred, gbm_pred, lgb_pred])
        pred = np.transpose(pred)

        rmsle_sum = self.xgb_rmsle + self.gbm_rmsle + self.lgb_rmsle

        self.xgb_per = self.xgb_rmsle / rmsle_sum
        self.gbm_per = self.gbm_rmsle / rmsle_sum
        self.lgb_per = self.lgb_rmsle / rmsle_sum
        
        final = xgb_pred*self.xgb_per + gbm_pred*self.gbm_per + lgb_pred*self.lgb_per
        print("--final RSMLE--")
        self.rmsle(self.y_test, final)

    # making the prediction template about 03/04/07/2020
    def make_temp(self):
        CARD_SIDO_NMs = self.encoding_data['CARD_SIDO_NM'].unique()
        STD_CLSS_NMs  = self.encoding_data['STD_CLSS_NM'].unique()
        HOM_SIDO_NMs  = self.encoding_data['HOM_SIDO_NM'].unique()
        AGEs          = self.encoding_data['AGE'].unique()
        SEX_CTGO_CDs  = self.encoding_data['SEX_CTGO_CD'].unique()
        FLCs          = self.encoding_data['FLC'].unique()
        nums          = self.encoding_data['num'].unique()
        coronas       = self.encoding_data['corona'].unique()
        # cates         = self.encoding_data['cate'].unique()
        years         = [2020]
        months        = [4, 7]

        comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, nums, coronas, years, months]
        temp = np.array(list(product(*comb_list)))

        train_features = self.encoding_data.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
        tmp = pd.DataFrame(data=temp, columns=train_features.columns)

        return tmp

    # making submission
    def make_sub(self, temp):
        xgb_pred = self.xgb_reg.predict(temp)
        xgb_pred = np.expm1(xgb_pred)

        gbm_pred = self.gbm_reg.predict(temp)
        gbm_pred = np.expm1(gbm_pred)

        lgb_pred = self.lgb_reg.predict(temp)
        lgb_pred = np.expm1(lgb_pred)

        final_rmsle = xgb_pred*self.xgb_per + gbm_pred*self.gbm_per + lgb_pred*self.lgb_per

        temp['AMT'] = np.round(final_rmsle, 0)
        temp['REG_YYMM'] = temp['year']*100 + temp['month']
        temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
        temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop = False)

        temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
        temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

        return temp

In [ ]:
def sampling(sido):
    df = data[data['CARD_SIDO_NM']==sido]
    df = df.sample(n=10000)
    return df

In [ ]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
def grap_corona(data):
  corona = [202002, 202003]
  
  if data in corona:
    return 1
  else :
    return 0

In [ ]:
def grap_cate(data):
  incr = ['골프장 운영업']
  des = ['관광 민예품 및 선물용품 소매업', '그외 기타 스포츠시설 운영업', '마사지업', '면세점', '버스 운송업', '여관업', '욕탕업', '전시 및 행사 대행업', '정기 항공 운송업', '호텔업', '휴양콘도 운영업', '일반유흥 주점업']

  if data in incr:
    return 1
  elif data in des:
    return -1
  else:
    return 0

In [ ]:
def data_pre(data):
    data = data.fillna('')
    data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
    data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
    data['corona'] = data['REG_YYMM'].apply(lambda x: grap_corona(x))
    # data['cate'] = data['STD_CLSS_NM'].apply(lambda x: grap_cate(x))
    data = data.drop(['REG_YYMM'], axis = 1)

    return data

In [ ]:
def encoding(data):

    global encoders
    dtypes = data.dtypes
    encoders = {}
    for column in data.columns:
        if str(dtypes[column]) == 'object':
            encoder = LabelEncoder()
            encoder.fit(data[column])
            encoders[column] = encoder

    data_num = data.copy()

    for column in encoders.keys():
        encoder = encoders[column]
        data_num[column] = encoder.transform(data[column])

    return data_num

In [ ]:
def split(data_num):
    split_data = []

    X_data, y_data = data_num.loc[:, data_num.columns != 'AMT'], data_num['AMT']
    X_data = X_data.drop(['CSTMR_CNT', 'CNT'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=126, shuffle=True)
    y_train = np.log1p(y_train)
    
    return X_train, X_test, y_train, y_test

In [ ]:
# train 데이터 불러오기
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/jeju/201901-202003.csv')
# 시도별 예측 데이터 프레임 생성을 위한 시도 리스트 생성
sido_list = train['CARD_SIDO_NM'].unique().tolist()

In [ ]:
data_2004 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/jeju/202004.csv')

In [ ]:
data_2003 = train[train['REG_YYMM'] == 202003]

In [ ]:
tmp03 = data_2003.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM']).AMT.sum().reset_index()
tmp04 = data_2004.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM']).AMT.sum().reset_index()
tmp = pd.merge(tmp03, tmp04, on = ['CARD_SIDO_NM', 'STD_CLSS_NM'])
tmp.columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT_03', 'AMT_04']
tmp

,CARD_SIDO_NM,STD_CLSS_NM,AMT_03,AMT_04
0,강원,건강보조식품 소매업,96059012,88823988
1,강원,골프장 운영업,2915797995,4708346820
2,강원,과실 및 채소 소매업,994816943,1121028924
3,강원,관광 민예품 및 선물용품 소매업,13317300,14360780
4,강원,그외 기타 스포츠시설 운영업,2075000,227200
...,...,...,...,...
599,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299,1373635928
600,충북,한식 음식점업,16152482704,18911036160
601,충북,호텔업,15248550,14121500
602,충북,화장품 및 방향제 소매업,428881434,450507431


In [ ]:
tmp['num'] = tmp.apply(lambda x: 1 if x['AMT_03'] < x['AMT_04'] else 0, axis = 1)
tmp

,CARD_SIDO_NM,STD_CLSS_NM,AMT_03,AMT_04,num
0,강원,건강보조식품 소매업,96059012,88823988,0
1,강원,골프장 운영업,2915797995,4708346820,1
2,강원,과실 및 채소 소매업,994816943,1121028924,1
3,강원,관광 민예품 및 선물용품 소매업,13317300,14360780,1
4,강원,그외 기타 스포츠시설 운영업,2075000,227200,0
...,...,...,...,...,...
599,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299,1373635928,1
600,충북,한식 음식점업,16152482704,18911036160,1
601,충북,호텔업,15248550,14121500,0
602,충북,화장품 및 방향제 소매업,428881434,450507431,1


In [ ]:
data = train.copy()

In [ ]:
data = data.merge(tmp[['CARD_SIDO_NM', 'STD_CLSS_NM', 'num']], how = 'left')

data

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,num
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,0.0
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,0.0
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,0.0
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,0.0
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4,0.0
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3,0.0
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6,0.0
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6,0.0


In [ ]:
data = data.fillna(0)
data['num'] = data['num'].apply(int)

In [ ]:
for sido in sido_list:
  temp = sampling(sido).reset_index(drop=True)

  temp = data_pre(temp)
  temp = temp.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

  temp_num = encoding(temp)
  temp_data = split(temp_num)

  temp = Model(temp_data, temp_num)

  temp.get_model()

  temp_temp = temp.make_temp()

  temp_sub = temp.make_sub(temp_temp)

  globals()['sub_{}'.format(sido)] = temp_sub

[09:15:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
from numpy import inf

def sido_diff(sido):


  df = train[train['CARD_SIDO_NM'] == sido]
  df_19 = df[df['REG_YYMM'] == 201903]
  df_20 = df[df['REG_YYMM'] == 202003]
  diff = pd.concat([df_19.groupby('STD_CLSS_NM').AMT.sum(), df_20.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys = ['1903_AMT', '2003_AMT']).reset_index()
  diff = diff.fillna(0)
  diff['diff'] = (diff['1903_AMT']-diff['2003_AMT'])/diff['1903_AMT']
  diff['diff'] = diff['diff'].apply(lambda x: 0 if x == -inf else x)
  diff.columns = ['STD_CLSS_NM', '1903_AMT', '2003_AMT', 'diff']
  diff = diff[['STD_CLSS_NM', 'diff']]
  diff = diff.sort_values(by='diff', ascending=False).reset_index(drop=True)

  return diff

In [ ]:
for sido in sido_list:
  tmp = sido_diff(sido)

  globals()['diff_{}'.format(sido)] = tmp

In [ ]:
test_subs = [sub_강원, sub_경기, sub_경남, sub_경북, sub_광주, sub_대구, sub_대전, sub_부산, sub_서울, sub_세종, sub_울산, sub_인천, sub_전남, sub_전북, sub_제주, sub_충남, sub_충북]
test_diffs = [diff_강원, diff_경기, diff_경남, diff_경북, diff_광주, diff_대구, diff_대전, diff_부산, diff_서울, diff_세종, diff_울산, diff_인천, diff_전남, diff_전북, diff_제주, diff_충남, diff_충북]

In [ ]:
subs = test_subs.copy()
diffs = test_diffs.copy()

In [ ]:
for i in range(len(sido_list)):
  subs[i] = subs[i].merge(diffs[i][['STD_CLSS_NM','diff']], how='left', on='STD_CLSS_NM')
  subs[i]['AMT'] = subs[i]['AMT']*(1-subs[i]['diff'])
  subs[i] = subs[i][['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [ ]:
test = pd.concat(subs)
test = test.sort_values(by = ['REG_YYMM', 'CARD_SIDO_NM']).reset_index(drop = True)
test

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,2020,강원,건강보조식품 소매업,1.569476e+09
1,2020,강원,골프장 운영업,2.970706e+09
2,2020,강원,과실 및 채소 소매업,7.630486e+08
3,2020,강원,관광 민예품 및 선물용품 소매업,8.739376e+07
4,2020,강원,그외 기타 스포츠시설 운영업,5.698750e+07
...,...,...,...,...
1217,2120,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.977207e+08
1218,2120,충북,한식 음식점업,6.098676e+08
1219,2120,충북,호텔업,1.908757e+08
1220,2120,충북,화장품 및 방향제 소매업,2.619258e+08


In [ ]:
def date_error(data):
  if data == 2020:
    return 202004
  else :
    return 202007

In [ ]:
test['REG_YYMM'] = test['REG_YYMM'].apply(lambda x: date_error(x))
test

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.569476e+09
1,202004,강원,골프장 운영업,2.970706e+09
2,202004,강원,과실 및 채소 소매업,7.630486e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,8.739376e+07
4,202004,강원,그외 기타 스포츠시설 운영업,5.698750e+07
...,...,...,...,...
1217,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.977207e+08
1218,202007,충북,한식 음식점업,6.098676e+08
1219,202007,충북,호텔업,1.908757e+08
1220,202007,충북,화장품 및 방향제 소매업,2.619258e+08


In [ ]:
project_id = 'jeju-bigquery-282708'
client = bigquery.Client(project = 'jeju-bigquery-282708')

submission = client.query('''
  SELECT 
    * 
  FROM `jeju-bigquery-282708.jeju_bigdata.submission` 
  ''').to_dataframe()

In [ ]:
final = submission.drop(['AMT'], axis=1)
final = final.merge(test, on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,1.569476e+09
1,1,202004,강원,골프장 운영업,2.970706e+09
2,2,202004,강원,과실 및 채소 소매업,7.630486e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,8.739376e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.977207e+08
1390,1390,202007,충북,한식 음식점업,6.098676e+08
1391,1391,202007,충북,호텔업,1.908757e+08
1392,1392,202007,충북,화장품 및 방향제 소매업,2.619258e+08


In [ ]:
final_nan = final[final['AMT'].isnull()].reset_index(drop=True)
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
1,14,202004,강원,면세점,NaN
2,28,202004,강원,자동차 임대업,NaN
3,30,202004,강원,정기 항공 운송업,NaN
4,35,202004,강원,택시 운송업,NaN
...,...,...,...,...,...
169,1347,202007,충남,택시 운송업,NaN
170,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,NaN
171,1368,202007,충북,버스 운송업,NaN
172,1381,202007,충북,자동차 임대업,NaN


In [ ]:
train_03 = train[train['REG_YYMM']==202003]
train_03.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260508 entries, 23437284 to 24697791
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   REG_YYMM      1260508 non-null  int64 
 1   CARD_SIDO_NM  1260508 non-null  object
 2   CARD_CCG_NM   1255817 non-null  object
 3   STD_CLSS_NM   1260508 non-null  object
 4   HOM_SIDO_NM   1260508 non-null  object
 5   HOM_CCG_NM    1253658 non-null  object
 6   AGE           1260508 non-null  object
 7   SEX_CTGO_CD   1260508 non-null  int64 
 8   FLC           1260508 non-null  int64 
 9   CSTMR_CNT     1260508 non-null  int64 
 10  AMT           1260508 non-null  int64 
 11  CNT           1260508 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 125.0+ MB


In [ ]:
amt_nan = []
for idx, x in final_nan.iterrows():
  amt = sum(train_03[(train_03['CARD_SIDO_NM']==x['CARD_SIDO_NM']) & (train_03['STD_CLSS_NM']==x['STD_CLSS_NM'])]['AMT'])
  amt_nan.append(amt)

In [ ]:
amt_nan = pd.Series(amt_nan)
final_nan['AMT'] = amt_nan

In [ ]:
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0
1,14,202004,강원,면세점,0
2,28,202004,강원,자동차 임대업,4607000
3,30,202004,강원,정기 항공 운송업,0
4,35,202004,강원,택시 운송업,0
...,...,...,...,...,...
169,1347,202007,충남,택시 운송업,0
170,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,0
171,1368,202007,충북,버스 운송업,0
172,1381,202007,충북,자동차 임대업,0


In [ ]:
final = final.dropna(subset=['AMT'])
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,1.569476e+09
1,1,202004,강원,골프장 운영업,2.970706e+09
2,2,202004,강원,과실 및 채소 소매업,7.630486e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,8.739376e+07
5,5,202004,강원,그외 기타 스포츠시설 운영업,5.698750e+07
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.977207e+08
1390,1390,202007,충북,한식 음식점업,6.098676e+08
1391,1391,202007,충북,호텔업,1.908757e+08
1392,1392,202007,충북,화장품 및 방향제 소매업,2.619258e+08


In [ ]:
final = pd.concat([final, final_nan])
final = final.sort_values('id').reset_index(drop=True)

In [ ]:
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,1.569476e+09
1,1,202004,강원,골프장 운영업,2.970706e+09
2,2,202004,강원,과실 및 채소 소매업,7.630486e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,8.739376e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.977207e+08
1390,1390,202007,충북,한식 음식점업,6.098676e+08
1391,1391,202007,충북,호텔업,1.908757e+08
1392,1392,202007,충북,화장품 및 방향제 소매업,2.619258e+08


In [ ]:
final['AMT'] = final['AMT'].astype(int)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394 entries, 0 to 1393
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1394 non-null   int64 
 1   REG_YYMM      1394 non-null   int64 
 2   CARD_SIDO_NM  1394 non-null   object
 3   STD_CLSS_NM   1394 non-null   object
 4   AMT           1394 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 54.6+ KB


In [ ]:
final.to_csv('/content/drive/My Drive/Colab Notebooks/jeju/submission_0730_corona.csv', encoding='utf-8-sig', index=False)